In [3]:
# install pydrake

# try:
#   import pydrake
#   import underactuated
# except ImportError:
# !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
# from jupyter_setup import setup_underactuated
# setup_underactuated()


In [ ]:
from pydrake.all import eq, MathematicalProgram, Solve, Variable

import matplotlib.pyplot as plt
import numpy as np
import importlib

from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator

from pydrake.all import (
    DiagramBuilder, Simulator
)

from pydrake.multibody.tree import (
    JointActuatorIndex
)

from pydrake.geometry import SceneGraph
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.trajectories import PiecewisePolynomial
from pydrake.solvers.snopt import SnoptSolver

import kinematic_constraints
import dynamics_constraints
importlib.reload(kinematic_constraints)
importlib.reload(dynamics_constraints)
from dynamics_constraints import (
  EvaluateDynamics
)

In [ ]:
# set up constants

N = 7      # number of collocation points
n_x = 9
n_u = 3

# where we start, where we go, mass?, 
# start position
r_0 = 3474.8       #radius of Mars, in km
alpha_0 = -23.45   #longitude degrees
beta_0 = -2.94     #latitude degrees
Vx_0 = 0
Vy_0 = 0
Vz_0 = 0
m_0 = 15234        #kg
phi_0 = 90         #pitch degrees
psi_0 = 0          #yaw degrees

# final position
r_N = 3474.8       #radius of Mars, in km
alpha_N = -23.45   #degrees
beta_N = -1.94     #degrees
Vx_0 = 0
Vy_0 = 0
Vz_0 = 0
phi_N = 90
psi_N = 0

# initial control input 
T_0 = 0
phiddot_0 = 0
psiddot_0 = 0

# final control input
T_N = 0
phiddot_N = 0
psiddot_N = 0

# constraint coefficients
r_min = 3474.8
r_max = 3474.8 + 30 #30km above Mars surface
phi_min = -20      #degree
phi_max = 90       #degree
psi_min = -180     #degree
psi_max = 180      #degree
phidot_min = -8    #deg/sec
phidot_max = 8     #deg/sec
psidot_min = -8    #deg/sec
psidot_max = 8     #deg/sec
T_min = 4671       #Newton
T_max = 43148      #Newton

initial_state = np.array([r_0,alpha_0,beta_0,Vx_0,Vy_0,Vz_0,m_0,phi_0,psi_0])
input_limit = np.array([[T_min,phidot_min,psidot_min],[T_max,phidot_max,psidot_max]])


m = 8321.09        #kg
g = 3.711          #m/s^2
Isp = 302.39       #s (effective specific impulse of the lander, used in mdot equation)


In [ ]:
# Initialize Mathematical Program and Add Variables: Need Check

prog = MathematicalProgram()
x = np.zeros((N,n_x),dtype="object")
u = np.zeros((N,n_u),dtype="object")
# t = np.zeros((N,1),dtype="object")


for i in range(N):
    x[i] = prog.NewContinuousVariables(n_x,"x_" + str(i))
    u[i] = prog.NewContinuousVariabels(n_u,"u_" + str(i))
#     t[i] = prog.NewContinuousVariables(1,"t_" + str(i))
t_land = prog.NewContinuousVariables(1,"t_land")

t0 = 0.0
timesteps = np.linspace(t0,t_land,N)


In [ ]:
# TODO: Add cost (sum of square of all thrusts): NEED CHECK
gz = (timesteps[i+1] - timesteps[i]) * (u[:,0].T.dot[[:,0]])
# gz = -x[N-1,6]
prog.AddQuadraticCost(gz)


In [ ]:
# TODO: Add dynamic constraint

In [ ]:
# TODO: Add kinematic constraint

In [ ]:
# TODO: Add input/state limitation as bounding box constraints: NEED CHECK

# height limit: 0 <= h <= hmax
prog.AddBoundingBoxConstraint(r_min,r_max,x[:,2])

# mass upper limit
prog.AddBoundingBoxConstraint(0,m_0,x[:,6])

# angular position input limit
prog.AddBoundingBoxConstraint(phi_min,phi_max,x[:,7])
prog.AddBoundingBoxConstraint(psi_min,psi_max,x[:,8])

# thrust input limit
prog.AddBoundingBoxConstraint(T_min,T_max,u[:,0])

# omega angular velocity state limits
prog.AddBoundingBoxConstraint(phidot_min,phidot_max,u[:,1])
prog.AddBoundingBoxConstraint(psidot_min,psidot_max,u[:,2])



In [ ]:
# TODO: Set up solver initial guess: NOT FINISHED

# state initial guess
xinit = np.zeros((N,n_x))


# input initial guess
uinit = np.zeros((N,n_u))
for i in range(n_u):
    uinit[:,i] = np.random.uniform(input_limit[0,i],input_limit[1,i],N)
prog.SetInitialGuess(u,uinit)


In [ ]:
# Set up solver: NEED CHECK

solver = SnoptSolver()
result = solver.Solve(prog)

# get solution of states and inputs from solver result
x_sol = result.GetSolution(x)
u_sol = result.GetSolution(u)
t_sol = result.GetSolution(t_land)

timesteps_sol = np.linspace(t0,t_sol,N)

# get xdot from x_sol and u_sol
xdot_sol = np.zeros(x_sol.shape)
for i in range(N):
    xdot_sol[i] = EvaluateDynamics(x_sol[i],u_sol[i])

# create trajectory by interpolating between collocation points using piecewise polynomial
x_traj = PiecewisePolynomial.CubicHermite(timesteps_sol,x_sol.T,xdot_sol.T)
u_traj = PiecewisePolynomial.FirstOrderHold(timesteps_sol,u_sol.T)



In [ ]:
# TODO: create LQR to follow the trajectory (use HW2)

In [ ]:
# TODO: simulate and animate path, create disturbance in simulation